In [1]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

#import argparse
#parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#parser.add_argument('--lr', default = 0.1, type=float, help='learning rate')
#parser.add_argument('--resume', '-r', action='store_true',
#                   help='resume from checkpoint')
#args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
resume = True # if resume from the saved file
learning_rate = 0.01

In [2]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root = './cifar10', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 200, shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root = './cifar10', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = 200, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Model
print('==> Building model..')
'''DLA in PyTorch.

Reference:
    Deep Layer Aggregation. https://arxiv.org/abs/1707.06484
'''
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Root(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 1):
        super(Root, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride = 1, padding = (kernel_size - 1) // 2, bias = False)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, xs):
        x = torch.cat(xs, 1)
        out = F.relu(self.bn(self.conv(x)))
        return out


class Tree(nn.Module):
    def __init__(self, block, in_channels, out_channels, level = 1, stride = 1):
        super(Tree, self).__init__()
        self.level = level
        if level == 1:
            self.root = Root(2 * out_channels, out_channels)
            self.left_node = block(in_channels, out_channels, stride = stride)
            self.right_node = block(out_channels, out_channels, stride = 1)
        else:
            self.root = Root((level + 2) * out_channels, out_channels)
            for i in reversed(range(1, level)):
                subtree = Tree(block, in_channels, out_channels, level = i, stride = stride)
                self.__setattr__('level_%d' % i, subtree)
            self.prev_root = block(in_channels, out_channels, stride = stride)
            self.left_node = block(out_channels, out_channels, stride = 1)
            self.right_node = block(out_channels, out_channels, stride = 1)

    def forward(self, x):
        xs = [self.prev_root(x)] if self.level > 1 else []
        for i in reversed(range(1, self.level)):
            level_i = self.__getattr__('level_%d' % i)
            x = level_i(x)
            xs.append(x)
        x = self.left_node(x)
        xs.append(x)
        x = self.right_node(x)
        xs.append(x)
        out = self.root(xs)
        return out


class DLA(nn.Module):
    def __init__(self, block = BasicBlock, num_classes = 10):
        super(DLA, self).__init__()
        self.base = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm2d(16),
            nn.ReLU(True)
        )

        self.layer1 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm2d(16),
            nn.ReLU(True)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm2d(32),
            nn.ReLU(True)
        )

        self.layer3 = Tree(block,  32,  64, level = 1, stride = 1)
        self.layer4 = Tree(block,  64, 128, level = 2, stride = 2)
        self.layer5 = Tree(block, 128, 256, level = 2, stride = 2)
        self.layer6 = Tree(block, 256, 512, level = 1, stride = 2)
        self.linear = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
# net = SimpleDLA()
net = DLA()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume: #args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('cifar10/checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./cifar10/checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = learning_rate,
                      momentum = 0.9, weight_decay = 5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 200)

==> Building model..
==> Resuming from checkpoint..


In [4]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if (batch_idx + 1) % 50 == 0:
            print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if (batch_idx + 1) % 50 == 0:
                print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    # Save checkpoint.
    acc = 100. * correct / total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('cifar10/checkpoint'):
            os.mkdir('cifar10/checkpoint')
        torch.save(state, './cifar10/checkpoint/ckpt.pth')
        best_acc = acc


for epoch in range(start_epoch, start_epoch + 20):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 34
49 250 Loss: 0.032 | Acc: 99.030% (9903/10000)
99 250 Loss: 0.037 | Acc: 98.785% (19757/20000)
149 250 Loss: 0.039 | Acc: 98.690% (29607/30000)
199 250 Loss: 0.040 | Acc: 98.647% (39459/40000)
249 250 Loss: 0.041 | Acc: 98.624% (49312/50000)
49 50 Loss: 0.245 | Acc: 93.190% (9319/10000)

Epoch: 35
49 250 Loss: 0.040 | Acc: 98.580% (9858/10000)
99 250 Loss: 0.039 | Acc: 98.630% (19726/20000)
149 250 Loss: 0.039 | Acc: 98.667% (29600/30000)
199 250 Loss: 0.039 | Acc: 98.680% (39472/40000)
249 250 Loss: 0.039 | Acc: 98.690% (49345/50000)
49 50 Loss: 0.240 | Acc: 93.230% (9323/10000)

Epoch: 36
49 250 Loss: 0.036 | Acc: 98.770% (9877/10000)
99 250 Loss: 0.040 | Acc: 98.630% (19726/20000)
149 250 Loss: 0.040 | Acc: 98.650% (29595/30000)
199 250 Loss: 0.040 | Acc: 98.698% (39479/40000)
249 250 Loss: 0.041 | Acc: 98.642% (49321/50000)
49 50 Loss: 0.260 | Acc: 92.670% (9267/10000)

Epoch: 37
49 250 Loss: 0.033 | Acc: 98.970% (9897/10000)
99 250 Loss: 0.034 | Acc: 98.905% (19781/2000

In [5]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2.1f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

print('Currently, the best total accuracy is %2.1f%%' % best_acc)

Accuracy of plane : 90.9 %
Accuracy of   car : 100.0 %
Accuracy of  bird : 94.7 %
Accuracy of   cat : 69.6 %
Accuracy of  deer : 88.9 %
Accuracy of   dog : 93.3 %
Accuracy of  frog : 81.8 %
Accuracy of horse : 100.0 %
Accuracy of  ship : 96.0 %
Accuracy of truck : 100.0 %
Currently, the best total accuracy is 93.6%
